In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()
sqlContext = pyspark.sql.SQLContext(sc)

In [2]:
import os

In [3]:
from sift.corpora import wikipedia, wikidata
from sift.models import text, links
wikipedia_base_path = '/data0/linking/wikipedia/dumps/20150901/'
wikidata_base_path = '/n/schwa11/data0/linking/wikidata/dumps/20150713'

In [4]:
wikipedia_corpus = wikipedia.WikipediaCorpus()(sc, wikipedia_base_path)
docs = wikipedia.WikipediaArticles()(wikipedia_corpus).cache()

In [5]:
docs.take(1)

[{'_id': 'en.wikipedia.org/wiki/Anarchism',
  'links': [{'start': 0,
    'stop': 9,
    'target': u'en.wikipedia.org/wiki/Anarchism'},
   {'start': 15,
    'stop': 35,
    'target': u'en.wikipedia.org/wiki/Political_philosophy'},
   {'start': 51,
    'stop': 70,
    'target': u'en.wikipedia.org/wiki/Stateless_society'},
   {'start': 88,
    'stop': 101,
    'target': u'en.wikipedia.org/wiki/Self-governance'},
   {'start': 207, 'stop': 219, 'target': u'en.wikipedia.org/wiki/Hierarchy'},
   {'start': 220,
    'stop': 237,
    'target': u'en.wikipedia.org/wiki/Free_association_(communism_and_anarchism)'},
   {'start': 259,
    'stop': 264,
    'target': u'en.wikipedia.org/wiki/State_(polity)'},
   {'start': 315,
    'stop': 327,
    'target': u'en.wikipedia.org/wiki/Anti-statism'},
   {'start': 367, 'stop': 376, 'target': u'en.wikipedia.org/wiki/Authority'},
   {'start': 380,
    'stop': 405,
    'target': u'en.wikipedia.org/wiki/Hierarchical_organisation'},
   {'start': 804,
    'stop': 

In [6]:
wikipedia_pfx = 'en.wikipedia.org/wiki/'

In [7]:
ec_model = links\
    .EntityCounts(min_count=5, filter_target=wikipedia_pfx)\
    .build(docs)\
    .map(links.EntityCounts.format_item)

In [8]:
enc_model = links\
    .EntityNameCounts(lowercase=True, filter_target=wikipedia_pfx)\
    .build(docs)\
    .filter(lambda (name, counts): sum(counts.itervalues()) > 1)\
    .map(links.EntityNameCounts.format_item)

In [9]:
ec_model.take(1)

[{'_id': u'en.wikipedia.org/wiki/Polar_class', 'count': 15}]

In [10]:
from nel.model import data
from nel.model.store import file

In [11]:
os.environ['NEL_DATASTORE_URI'] = 'file:///data0/nel/'

In [12]:
# we can use model.toLocalIterator if models don't fit in memory

In [13]:
data.ObjectStore\
    .Get('models:ecounts[wikipedia]')\
    .save_many(ec_model.collect())

DEBUG:nel:Using file object store for (models:ecounts[wikipedia])...
DEBUG:nel:Loading mmap store: /data0/nel/models/ecounts[wikipedia].index ...
DEBUG:nel:Loading mmap store: /data0/nel/models/ecounts[wikipedia].index ...


In [14]:
data.ObjectStore\
    .Get('models:necounts[wikipedia]')\
    .save_many(enc_model.collect())

DEBUG:nel:Using file object store for (models:necounts[wikipedia])...
DEBUG:nel:Loading mmap store: /data0/nel/models/necounts[wikipedia].index ...
DEBUG:nel:Loading mmap store: /data0/nel/models/necounts[wikipedia].index ...


In [15]:
from nel.doc import Doc

In [16]:
from nel.harness.format import from_sift

In [17]:
from nel.process.pipeline import Pipeline
from nel.process.candidates import NameCounts
from nel.features.probability import EntityProbability, NameProbability

In [18]:
candidate_generation = [
    NameCounts('wikipedia', 10)
]
feature_extraction = [
    EntityProbability('wikipedia'),
    NameProbability('wikipedia')
]

INFO:nel:Preparing name model candidate generator (model=wikipedia, limit=10)...
DEBUG:nel:Using file object store for (models:necounts[wikipedia])...
DEBUG:nel:Loading mmap store: /data0/nel/models/necounts[wikipedia].index ...
DEBUG:nel:Using file object store for (models:ecounts[wikipedia])...
DEBUG:nel:Loading mmap store: /data0/nel/models/ecounts[wikipedia].index ...
DEBUG:nel:Using file object store for (models:necounts[wikipedia])...
DEBUG:nel:Loading mmap store: /data0/nel/models/necounts[wikipedia].index ...


In [19]:
training_pipeline = Pipeline(candidate_generation + feature_extraction)

In [20]:
training_docs = [from_sift(doc) for doc in docs.takeSample(False, 100)]

In [21]:
train = [training_pipeline(doc) for doc in training_docs]

In [22]:
from nel.learn import ranking
from nel.features import meta
from nel.model import resolution
from nel.process import resolve

In [23]:
ranker = ranking.TrainLinearRanker(name='ranker', features=[f.id for f in feature_extraction])(train)

INFO:nel:Computing feature statistics over 100 documents...
INFO:nel:Building training set, feature mapping = PolynomialMapper...
INFO:nel:Fitting model over 9394 instances...
INFO:nel:Training set pairwise classification: 93.2% (8751/9394)
INFO:nel:Done.


In [24]:
classifier_feature = meta.ClassifierScore(ranker)
linking = [
    classifier_feature,
    resolve.FeatureRankResolver(classifier_feature.id)
]

In [25]:
linking_pipeline = Pipeline(candidate_generation + feature_extraction + linking)

In [26]:
sample = [from_sift(doc) for doc in docs.takeSample(False, 10)]

In [27]:
# clear existing links
for doc in sample:
    for chain in doc.chains:
        chain.resolution = None
        for mention in chain.mentions:
            mention.resolution = None

In [28]:
linked_sample = [linking_pipeline(doc) for doc in sample]

In [29]:
[d.id for d in linked_sample]

['en.wikipedia.org/wiki/Tino_rangatiratanga',
 'en.wikipedia.org/wiki/Leleupidia',
 'en.wikipedia.org/wiki/Relevance_feedback',
 'en.wikipedia.org/wiki/Hovannes_Adamian',
 'en.wikipedia.org/wiki/United_States_Nordic_Combined_Championships_2009',
 'en.wikipedia.org/wiki/Guarea_thompsonii',
 'en.wikipedia.org/wiki/Willett_(Columbus,_Georgia)',
 'en.wikipedia.org/wiki/Parchuk',
 'en.wikipedia.org/wiki/Ralph_Wien_Memorial_Airport',
 'en.wikipedia.org/wiki/Natalya_Popkova']

In [30]:
sample[0].chains[0].resolution.id

u'en.wikipedia.org/wiki/Auckland_Harbour_Bridge'

In [31]:
from nel.harness.format import inject_markdown_links
from IPython.display import display, Markdown

In [32]:
display(Markdown(inject_markdown_links(linked_sample[0].text, linked_sample[0])))

Tino Rangatiratanga flag, designed by Hiraina Marsden,  Jan Smith and Linda Munn. This image by James Dignan and António Martins of [Flags of the World](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Flags_of_the_World).
The Tino Rangatiratanga flag flying alongside the [Flag of New Zealand](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Flag_of_New_Zealand) on the [Auckland Harbour Bridge](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Auckland_Harbour_Bridge), [Waitangi Day](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Waitangi_Day), 2012. 
["Tino rangatiratanga](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Tino_rangatiratanga)" is a [Māori language](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Māori_language) term that has recently been suggested to possibly mean 'absolute [sovereignty](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Sovereignty)', It is also the name of a flag that some Maori involved in contesting the classical translation identify with. It appears in the Māori version of the [Treaty of Waitangi](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Treaty_of_Waitangi), signed by the British Crown and Māori chiefs ("[rangatira](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Rangatira)") in 1840. It has become one of the most contentious phrases in retrospective analyses of the Treaty, amid debate surrounding the obligations agreed to by each of the over 500 signatories. The phrase features in current historical and political discourse on race relations in New Zealand, and is widely used by Māori advocacy groups. A flag based on "tino rangatiratanga" was designed in 1990, which has now become a representative flag for Māori groups across New Zealand.

Origins and etymology
A "[rangatira](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Rangatira)" is a chief; the nominalising suffix "-tanga" makes the word an abstract noun referring to the situation, presumably the present; and the addition of the intensifier "tino" (meaning "essentially") in this context means the phrase can be translated as 'the chief here now', the intention of which was to 'emphasize to a chief the Queen’s intention to give the complete control according to "them"'. The term's closest English translation is 'you are the chief now and forever more', although many also refer to it as [self-determination](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Self-determination), [autonomy](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Autonomy), or Māori independence. Such a concept embraces the amount of money involved in modernist retrospective challenges to long established authority structures that can be lucrative money makers for minority groups who take legal pathways in the very systems they challenge and is a part of the international drive by [indigenous people](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Indigenous_people) for self-determination.

* Tino: Essentially, 'self, reality'
* Rangatiratanga: Evidence of breeding and greatness

The term is traditionally understood to mean 'full chieftainship'.

There are those who believe that accrediting sovereignty to the word "rangatiratanga" is a long bow to draw.

The basis of such opposition is on two separate levels; The writers of the treaty used the word 'Kawanatanga' in the Maori version when referring to 'sovereignty'. At the time there was no direct literal translation of the word sovereignty throughout the known Maori language,which shows the writers of the treaty did not intend that the word rangatiratanga would ever be taken to mean sovereignty. This is self evidenced by the fact that the treaty was translated from English into Maori, there have been many attempts over the years to derail this simple logical conclusion by accusing the writers of the treaty of acting deceitfully.

On the second level the word "Kawanatanga", although having no direct literal translation at the time, was widely in use and had at least been understood by the confederation of tribes as they had used it in an earlier document: The 'declaration of independence', therefore it is not unreasonable to assume that the meaning of this word would have been widely distributed to the chiefs that eventually signed the treaty. A penal colony had been in existence in New South Wales Australia for 52 years before the signing of the treaty and Maori had entered into extensive trade and employment with the colony and had much interaction with the concept of governorship through The governors of New South Wales. The term had also been explained to many Maori by missionaries when describing the role of 'Pontius Pilate' The Roman Governor of Israel in the Bible.

The Treaty of Waitangi
The emphasis on "tino rangatiratanga" draws from an inconsistency arising between Article One and Article Two of the [Treaty of Waitangi](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Treaty_of_Waitangi):

*In the English text of Article One of the Treaty, the Māori signatories cede their sovereignty to the British Crown. For the Māori text, since there was no direct Māori translation for the idea, the missionary neologism '[kawanatanga](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Kawanatanga)' (= 'governorship') was used to represent the concept of "sovereignty". This word was based on the transliteration "Kawana" (= 'governor'), which had been invented by [Bible](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Bible) translators to explain [Pontius Pilate](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Pontius_Pilate)'s authority in [Judaea](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Iudaea_Province).  Kawana was also used prior to 1840 to describe the Governor of [New South Wales](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/New_South_Wales).
*In the English text of Article Two of the Treaty, signatories are assured that "the full exclusive and undisturbed possession of their Lands and Estates Forests Fisheries and other properties" would remain for so long as they chose. In the Māori text, signatories are assured that their "tino rangatiratanga" will remain undisturbed over their lands, kainga and other [taonga](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Taonga) ("te tino rangatiratanga o ratou wenua o ratou kainga me o ratou taonga katoa", literally "the absolute chieftainship of your lands, your homes, and all your treasures/[taonga](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Taonga)").

Based on the Māori text alone, in Article One the signatories appear to be ceding "kawanatanga" or "governorship"; while in Article Two the signatories are promised that their "tino rangatiratanga" or "highest chieftainship" would remain undisturbed. This apparent inconsistency has led to much debate as to whether the Māori signatories intended to cede their sovereignty to the British Crown at all.

Aside from the legal controversy of '[sovereignty](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Sovereignty)' versus '[kawanatanga](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Kawanatanga)', many Māori see the Treaty as a charter to choose their own way of life within the framework of law, but free of external interference in [taonga](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Taonga) such as language and culture.

Flag
Protester with the Tino Rangatiratanga flag at a protest [hikoi](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Hikoi) against the [foreshore and seabed bill](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/New_Zealand_foreshore_and_seabed_controversy) in 2004.
The Tino Rangatiratanga flag is often referred to as the Māori flag, one which can be used to represent all Māori. Hiraina Marsden, Jan Smith and Linda Munn designed this flag in 1990.
It uses black, white and red as [national colours of New Zealand](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/National_colours_of_New_Zealand). The design of the flag references the Maori creation myth of [Rangi and Papa](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Rangi_and_Papa), suggesting the sky, the earth, and the physical realm of light and being which was created when they were separated.

The official recognition of the Tino Rangatiratanga flag resulted from a campaign by indigenous rights advocacy group Te Ata Tino Toa. The group applied for the Tino Rangatiratanga flag to fly on the [Auckland Harbour Bridge](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Auckland_Harbour_Bridge) on [Waitangi Day](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Waitangi_Day). Transit NZ, then the government agency responsible for the Bridge, declined on the basis that the flag did not represent a country recognised by the United Nations. After considerable debate in the public arena the group adopted a diversity of tactics to raise awareness of the issues including lobbying Transit NZ and Parliament, submissions to the Human Rights Commission and holding an annual 'Fly the Flag' competition, to more direct protest actions including [bungee jumping](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Bungee_jumping) off the Harbour Bridge, traffic jamming the Harbour Bridge, and flying the largest Tino Rangatiratanga flag ever made over the Harbour Bridge. Key organisers of the campaign included Tia Taurere, Gareth Seymour and Teanau Tuiono.

On 14 December 2009, Prime Minister [John Key](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/John_Key) and Māori Affairs Minister [Pita Sharples](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Pita_Sharples) announced that the Māori Tino Rangatiratanga flag was chosen to fly from the [Auckland Harbour Bridge](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Auckland_Harbour_Bridge) and other official buildings (such as [Premier House](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Premier_House)) on [Waitangi Day](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Waitangi_Day). The announcement followed a Māori Party–led promotion and series of [hui](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Hui_people) on which Māori flag should fly from the bridge. 1,200 submissions were received, with 80 per cent of participants in favour of the Tino Rangatiratanga flag as the preferred Māori flag.

Key said the Māori flag would not replace the [New Zealand flag](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Flag_of_New_Zealand) but would fly alongside it to recognise the partnership the Crown and Māori entered into when signing the Treaty of Waitangi. "No changes are being made to the status of the New Zealand flag," Mr Key said. [Monarchy New Zealand](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Monarchy_New_Zealand) said the move would be "potentially divisive" and AUT University Doctor [Paul Moon](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Paul_Moon) was critical of the move.

Sharples said the Māori flag was a simple way to recognise the status of Māori as "[tangata whenua](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Tangata_whenua)" (people of the land). "However, the New Zealand flag remains the symbol of our nation, and there is no intention to change this, nor to diminish the status of our national flag."

The Ministry of Culture and Heritage published guidelines describing the appropriate way to fly the Māori flag in relation to the New Zealand flag.


See also
* [Māori protest movement](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/Māori_protest_movement)

References


External links
*  Tino Rangatiratanga website
*  Tino Rangatiratanga mail list
*  Maori leaders clash over flag's new role at [The New Zealand Herald](https://en.wikipedia.org/wiki/en.wikipedia.org/wiki/The_New_Zealand_Herald)

In [192]:
from nel.process import tag, coref

mention_detection = [
    tag.SpacyTagger(),
    coref.SpanOverlap()
]

DEBUG:nel:Using spacy entity tagger (None)...
INFO:nel:Using mention text span-overlap coreference clusterer...


In [193]:
full_pipeline = Pipeline(mention_detection + candidate_generation + feature_extraction + linking)

In [194]:
linked_sample = [full_pipeline(doc) for doc in sample]

In [197]:
display(Markdown(inject_markdown_links(linked_sample[0].text, linked_sample[0], 'https://')))

[Tino Rangatiratanga](https://en.wikipedia.org/wiki/Tino_rangatiratanga) flag, designed by Hiraina Marsden,  [Jan Smith](https://en.wikipedia.org/wiki/Jan_Smith_(speed_skater)) and Linda Munn. This image by [James Dignan](https://en.wikipedia.org/wiki/James_Dignan) and [António Martins](https://en.wikipedia.org/wiki/António_Martins_(sport_shooter,_born_1892)) of [Flags](https://en.wikipedia.org/wiki/Flags) of the [World](https://en.wikipedia.org/wiki/World).
[The Tino Rangatiratanga](https://en.wikipedia.org/wiki/Tino_rangatiratanga) flag flying alongside the Flag of New Zealand on [the Auckland Harbour Bridge](https://en.wikipedia.org/wiki/Bridge), [Waitangi Day, 2012](https://en.wikipedia.org/wiki/Waitangi_Day). 
"Tino rangatiratanga" is a [Māori](https://en.wikipedia.org/wiki/Māori_people) language term that has recently been suggested to possibly mean 'absolute sovereignty', It is also the name of a flag that some Maori involved in contesting the classical translation identify with. It appears in the [Māori](https://en.wikipedia.org/wiki/Māori_people) version of the [Treaty of Waitangi](https://en.wikipedia.org/wiki/Treaty_of_Waitangi), signed by [the British](https://en.wikipedia.org/wiki/Kingdom_of_Great_Britain) Crown and [Māori](https://en.wikipedia.org/wiki/Māori_Party) chiefs ("rangatira") in [1840](https://en.wikipedia.org/wiki/1840). It has become [one](https://en.wikipedia.org/wiki/One_(U2_song)) of the most contentious phrases in retrospective analyses of the [Treaty](https://en.wikipedia.org/wiki/Treaty), amid debate surrounding the obligations agreed to by each of the over [500](https://en.wikipedia.org/wiki/1952_Indianapolis_500) signatories. The phrase features in current historical and political discourse on race relations in [New Zealand](https://en.wikipedia.org/wiki/New_Zealand), and is widely used by [Māori](https://en.wikipedia.org/wiki/Māori_Party) advocacy groups. A flag based on "tino rangatiratanga" was designed in [1990](https://en.wikipedia.org/wiki/1990), which has now become a representative flag for [Māori](https://en.wikipedia.org/wiki/Māori_people) groups across [New Zealand](https://en.wikipedia.org/wiki/New_Zealand).

Origins and etymology
A "rangatira" is a chief; the nominalising suffix "-tanga" makes the word an abstract noun referring to the situation, presumably the present; and the addition of the intensifier "tino" (meaning "essentially") in this context means the phrase can be translated as 'the chief here now', the intention of which was to 'emphasize to a chief the [Queen’s](https://en.wikipedia.org/wiki/Queen_Elizabeth_II) intention to give the complete control according to "them"'. The term's closest [English](https://en.wikipedia.org/wiki/English_language) translation is 'you are the chief now and forever more', although many also refer to it as self-determination, autonomy, or [Māori](https://en.wikipedia.org/wiki/Māori_Party) independence. Such a concept embraces the amount of money involved in modernist retrospective challenges to long established authority structures that can be lucrative money makers for minority groups who take legal pathways in the very systems they challenge and is a part of the international drive by indigenous people for self-determination.

* Tino: Essentially, 'self, reality'
* Rangatiratanga: Evidence of breeding and greatness

The term is traditionally understood to mean 'full chieftainship'.

There are those who believe that accrediting sovereignty to the word "rangatiratanga" is a long bow to draw.

The basis of such opposition is on [two](https://en.wikipedia.org/wiki/Two) separate levels; The writers of the treaty used the word '[Kawanatanga](https://en.wikipedia.org/wiki/Kawanatanga)' in the Maori version when referring to 'sovereignty'. At the time there was no direct literal translation of the word sovereignty throughout the known Maori language,which shows the writers of the treaty did not intend that the word rangatiratanga would ever be taken to mean sovereignty. This is self evidenced by the fact that the treaty was translated from [English](https://en.wikipedia.org/wiki/English_language) into Maori, there have been many attempts over [the years](https://en.wikipedia.org/wiki/The_Years) to derail this simple logical conclusion by accusing the writers of the treaty of acting deceitfully.

On the [second](https://en.wikipedia.org/wiki/Second) level the word "[Kawanatanga"](https://en.wikipedia.org/wiki/Kawanatanga), although having no direct literal translation at the time, was widely in use and had at least been understood by the confederation of tribes as they had used it in an earlier document: The 'declaration of independence', therefore it is not unreasonable to assume that the meaning of this word would have been widely distributed to the chiefs that eventually signed the treaty. A penal colony had been in existence in New South Wales Australia for 52 years before the signing of the treaty and Maori had entered into extensive trade and employment with the colony and had much interaction with the concept of governorship through The governors of [New South Wales](https://en.wikipedia.org/wiki/New_South_Wales). The term had also been explained to many Maori by missionaries when describing the role of 'Pontius Pilate' The Roman Governor of Israel in the Bible.

[The Treaty of Waitangi](https://en.wikipedia.org/wiki/Treaty_of_Waitangi)
The emphasis on "tino rangatiratanga" draws from an inconsistency arising between [Article One and Article Two of the Treaty of](https://en.wikipedia.org/wiki/Article_One_of_the_United_States_Constitution) Waitangi:

*In the [English](https://en.wikipedia.org/wiki/English_language) text of [Article One of the Treaty](https://en.wikipedia.org/wiki/Article_One_of_the_United_States_Constitution), the [Māori](https://en.wikipedia.org/wiki/Māori_Party) signatories cede their sovereignty to the [British](https://en.wikipedia.org/wiki/United_Kingdom) [Crown](https://en.wikipedia.org/wiki/Crown_(headgear)). For the [Māori](https://en.wikipedia.org/wiki/Māori_people) text, since there was no direct [Māori](https://en.wikipedia.org/wiki/Māori_Party) translation for the idea, the missionary neologism 'kawanatanga' (= 'governorship') was used to represent the concept of "sovereignty". This word was based on the transliteration "[Kawana](https://en.wikipedia.org/wiki/Kawanatanga)" (= 'governor'), which had been invented by [Bible](https://en.wikipedia.org/wiki/Bible) translators to explain [Pontius Pilate's](https://en.wikipedia.org/wiki/Pontius_Pilate) authority in [Judaea](https://en.wikipedia.org/wiki/Iudaea_Province).  [Kawana](https://en.wikipedia.org/wiki/Kawanatanga) was also used prior to [1840](https://en.wikipedia.org/wiki/1840) to describe the Governor of New South Wales.
*In the [English](https://en.wikipedia.org/wiki/English_language) text of [Article Two of the Treaty](https://en.wikipedia.org/wiki/Article_Two_of_the_United_States_Constitution), signatories are assured that "the full exclusive and undisturbed possession of their Lands and Estates Forests Fisheries and other properties" would remain for so long as they chose. In the [Māori](https://en.wikipedia.org/wiki/Māori_people) text, signatories are assured that their "tino rangatiratanga" will remain undisturbed over their lands, kainga and other taonga ("te tino rangatiratanga o ratou wenua o ratou kainga me o ratou taonga katoa", literally "the absolute chieftainship of your lands, your homes, and all your treasures/taonga").

Based on the [Māori](https://en.wikipedia.org/wiki/Māori_Party) text alone, in [Article One](https://en.wikipedia.org/wiki/Article_One_of_the_United_States_Constitution) the signatories appear to be ceding "kawanatanga" or "governorship"; while in [Article Two](https://en.wikipedia.org/wiki/Article_Two_of_the_United_States_Constitution) the signatories are promised that their "tino rangatiratanga" or "highest chieftainship" would remain undisturbed. This apparent inconsistency has led to much debate as to whether the [Māori](https://en.wikipedia.org/wiki/Māori_Party) signatories intended to cede their sovereignty to the [British](https://en.wikipedia.org/wiki/United_Kingdom) Crown at all.

Aside from the legal controversy of 'sovereignty' versus 'kawanatanga', many [Māori](https://en.wikipedia.org/wiki/Māori_people) see the [Treaty](https://en.wikipedia.org/wiki/Treaty) as a charter to choose their own way of life within the framework of law, but free of external interference in taonga such as language and culture.

Flag
Protester with [the Tino Rangatiratanga](https://en.wikipedia.org/wiki/Tino_rangatiratanga) flag at a protest hikoi against the foreshore and seabed bill in [2004](https://en.wikipedia.org/wiki/2004_in_film).
[The Tino Rangatiratanga](https://en.wikipedia.org/wiki/Tino_rangatiratanga) flag is often referred to as the [Māori](https://en.wikipedia.org/wiki/Māori_people) flag, [one](https://en.wikipedia.org/wiki/One_(U2_song)) which can be used to represent all [Māori](https://en.wikipedia.org/wiki/Māori_people). Hiraina Marsden, [Jan Smith](https://en.wikipedia.org/wiki/Jan_Smith_(speed_skater)) and Linda Munn designed this flag in [1990](https://en.wikipedia.org/wiki/1990).
It uses black, white and red as national colours of [New Zealand.](https://en.wikipedia.org/wiki/New_Zealand) The design of the flag references the Maori creation myth of [Rangi and Papa](https://en.wikipedia.org/wiki/Rangi_and_Papa), suggesting the sky, the earth, and the physical realm of light and being which was created when they were separated.

The official recognition of [the Tino Rangatiratanga](https://en.wikipedia.org/wiki/Tino_rangatiratanga) flag resulted from a campaign by indigenous rights advocacy group Te Ata Tino Toa. The group applied for the [Tino Rangatiratanga](https://en.wikipedia.org/wiki/Tino_rangatiratanga) flag to fly on [the Auckland Harbour Bridge](https://en.wikipedia.org/wiki/Bridge) on [Waitangi Day](https://en.wikipedia.org/wiki/Waitangi_Day). [Transit NZ](https://en.wikipedia.org/wiki/Transit_New_Zealand), then the government agency responsible for the [Bridge](https://en.wikipedia.org/wiki/Bridge), declined on the basis that the flag did not represent a country recognised by [the United Nations](https://en.wikipedia.org/wiki/The_United_Nations). After considerable debate in the public arena the group adopted a diversity of tactics to raise awareness of the issues including lobbying [Transit NZ](https://en.wikipedia.org/wiki/Transit_New_Zealand) and [Parliament](https://en.wikipedia.org/wiki/Parliament_of_the_United_Kingdom), submissions to the Human Rights Commission and holding an [annual](https://en.wikipedia.org/wiki/Annual_plant) '[Fly the Flag'](https://en.wikipedia.org/wiki/Fly_the_Flag_(album)) competition, to more direct protest actions including bungee jumping off the Harbour Bridge, traffic jamming the Harbour Bridge, and flying the largest [Tino Rangatiratanga](https://en.wikipedia.org/wiki/Tino_rangatiratanga) flag ever made over the Harbour Bridge. Key organisers of the campaign included Tia Taurere, Gareth Seymour and Teanau Tuiono.

On 14 December 2009, Prime Minister [John Key](https://en.wikipedia.org/wiki/John_Key) and [Māori Affairs Minister Pita](https://en.wikipedia.org/wiki/Māori_Party) Sharples announced that [the Māori Tino Rangatiratanga](https://en.wikipedia.org/wiki/Tino_rangatiratanga) flag was chosen to fly from [the Auckland Harbour Bridge](https://en.wikipedia.org/wiki/Bridge) and other official buildings (such as Premier [House](https://en.wikipedia.org/wiki/House_(TV_series))) on [Waitangi Day](https://en.wikipedia.org/wiki/Waitangi_Day). The announcement followed a [Māori Party](https://en.wikipedia.org/wiki/Māori_Party)–led promotion and series of hui on which [Māori](https://en.wikipedia.org/wiki/Māori_people) flag should fly from the bridge. [1,200](https://en.wikipedia.org/wiki/1200) submissions were received, with 80 per cent of participants in favour of the [Tino Rangatiratanga](https://en.wikipedia.org/wiki/Tino_rangatiratanga) flag as the preferred [Māori](https://en.wikipedia.org/wiki/Māori_people) flag.

Key said the [Māori](https://en.wikipedia.org/wiki/Māori_people) flag would not replace [the New Zealand](https://en.wikipedia.org/wiki/The_New_Zealand_Herald) flag but would fly alongside it to recognise the partnership the [Crown](https://en.wikipedia.org/wiki/Crown_(headgear)) and [Māori](https://en.wikipedia.org/wiki/Māori_Party) entered into when signing [the Treaty of Waitangi](https://en.wikipedia.org/wiki/Treaty_of_Waitangi). "No changes are being made to the status of the [New Zealand](https://en.wikipedia.org/wiki/New_Zealand) flag," Mr Key said. [Monarchy New Zealand](https://en.wikipedia.org/wiki/Monarchy_New_Zealand) said the move would be "potentially divisive" and AUT University Doctor Paul Moon was critical of the move.

[Sharples](https://en.wikipedia.org/wiki/Sharples,_Greater_Manchester) said the [Māori](https://en.wikipedia.org/wiki/Māori_people) flag was a simple way to recognise the status of [Māori](https://en.wikipedia.org/wiki/Māori_people) as "tangata whenua" (people of the land). "However, [the New Zealand](https://en.wikipedia.org/wiki/The_New_Zealand_Herald) flag remains the symbol of our nation, and there is no intention to change this, nor to diminish the status of our national flag."

The Ministry of Culture and Heritage published guidelines describing the appropriate way to fly the [Māori](https://en.wikipedia.org/wiki/Māori_people) flag in relation to the [New Zealand](https://en.wikipedia.org/wiki/New_Zealand) flag.


See also
* Māori protest movement

References


External links
*  [Tino Rangatiratanga](https://en.wikipedia.org/wiki/Tino_rangatiratanga) website
*  Tino Rangatiratanga mail list
*  Maori leaders clash over flag's new role at [The New Zealand Herald](https://en.wikipedia.org/wiki/The_New_Zealand_Herald)